<a href="https://colab.research.google.com/github/soorower/Auto-Headers-Maker/blob/main/realforeclose_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output 
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
clear_output()

In [3]:
# first install gcsfuse
%%capture
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt update
!apt install gcsfuse



import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

#you can install module with below format
!pip install --target=$nb_path selenium

In [16]:
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup as bs
import sys
import json
import requests
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
driver.maximize_window()



headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}

listan = []
datan  ={}
def auction_scrape(date,types,left_side,right_side,county):
    l_rows = left_side.findAll('div')
    if l_rows[0].text == 'Auction Sold':
        auction_status = 'Sold at '+ l_rows[1].text.strip()
        sold_amount = l_rows[3].text.strip()
        sold_to = l_rows[5].text.strip()
    elif l_rows[0].text == 'Auction Starts':
        auction_status = 'Starts at '+ l_rows[1].text.strip()
        sold_amount = ''
        sold_to = ''
    else:
        auction_status = l_rows[1].text.strip()
        sold_amount = ''
        sold_to = ''
    #---------------------------------------------------------------------
    try:
        r_rows = right_side.find('table').find('tbody').findAll('tr')
        row_count = 0
        for row in r_rows:
            if row.find('th').text == 'Auction Type:':
                auction_type = row.find('td').text.strip()
            elif row.find('th').text == 'Case #:':
                case_hash = row.find('td').text.strip()
            elif row.find('th').text == 'Final Judgment Amount:':
                final_jud_amount = row.find('td').text.strip()
            elif row.find('th').text == 'Parcel ID:':
                parcel_id = row.find('td').text.strip()
            elif row.find('th').text == 'Property Address:':
                property_address = row.find('td').text.strip()
                if r_rows[row_count+1].find('th').text == '':
                    property_address = property_address +' '+ r_rows[row_count+1].find('td').text.strip()
            elif row.find('th').text == 'Assessed Value:':
                assessed_value = row.find('td').text.strip()
            elif row.find('th').text == 'Plaintiff Max Bid:':
                plaintiff_max_bid = row.find('td').text.strip()
            row_count = row_count + 1
    except:
        r_rows = right_side.find('div',attrs = {'class':'ad_tab'}).findAll('div',attrs = {'class':'AD_DTA'})
        auction_type = r_rows[0].text.strip()
        case_hash = r_rows[1].text.strip()
        final_jud_amount = r_rows[2].text.strip()
        parcel_id = r_rows[3].text.strip()
        try:
            property_address = r_rows[4].text.strip()
        except:
            property_address = ''
        try:
            assessed_value = r_rows[5].text.strip()
        except:
            assessed_value = ''
        try:
            plaintiff_max_bid = r_rows[6].text.strip()
        except:
            plaintiff_max_bid = ''
        
        
    try:
        auction_type = auction_type
    except:
        auction_type = ''
    try:
        case_hash = case_hash
    except:
        case_hash = ''
    try:
        final_jud_amount = final_jud_amount
    except:
        final_jud_amount = ''
    
    try:
        parcel_id = parcel_id
    except:
        parcel_id = ''
    try:
        property_address = property_address
    except:
        property_address = ''
    try:
        assessed_value = assessed_value
    except:
        assessed_value = ''
    try:
        plaintiff_max_bid = plaintiff_max_bid
    except:
        plaintiff_max_bid = ''

    #----------geting owner name-------------------------------------------------------------------
    new_parcel_id = parcel_id.replace('-','')
    if county == 'duval':
        owner_find_url = f'https://paopropertysearch.coj.net/Basic/Detail.aspx?RE={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')
        try:
            owner_name = soup.find('div',attrs= {'id':'ownerName'}).find('h2').text.strip()
        except:
            owner_name = ''
    elif county == 'miamidade':
        owner_find_url = f'https://www.miamidade.gov/Apps/PA/PApublicServiceProxy/PaServicesProxy.ashx?Operation=GetPropertySearchByFolio&clientAppName=PropertySearch&folioNumber={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')

        try:
            own_n= str(soup).find('"Name":"')
            owner_name = str(soup)[own_n+8:own_n+50].split('"')[0].strip()
            if 'ionals'==owner_name:
                owner_name = ''
        except:
            owner_name = ''
    elif county == 'broward':
        owner_find_url = f'https://bcpa.net/RecInfo.asp?URL_Folio={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')

        try:
            owner_name = soup.findAll('span',attrs = {'class':'BodyCopyBold9'})[1].text.strip()
        except:
            owner_name = ''
    elif county == 'hillsborough':
        owner_find_url = f'https://gis.hcpafl.org/CommonServices/property/search//ParcelData?pin={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')

        try:
            own_n= str(soup).find('"owner":"')
            owner_name = str(soup)[own_n+9:own_n+50].split('"')[0].strip()
        except:
            owner_name = ''
    elif county == 'orange': 
        owner_find_url = f'https://ocpa-mainsite-afd-standard.azurefd.net/api/PRC/GetPRCGeneralInfo?pid={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')

        try:
            own_n= str(soup).find('"ownerName":"')
            owner_name = str(soup)[own_n+13:own_n+50].split('"')[0].strip()
            if 'ps://tools.ietf.org/html/rfc7231#sect' == owner_name:
                owner_name = ''
            if 'pid'== owner_name:
                owner_name = ''
        except:
            owner_name = ''
    elif county == 'pinellas': 
        try:
            new_parcel = parcel_id.split('-',3)
            new_parcel_id = new_parcel[2] + new_parcel[1] +new_parcel[0] + new_parcel[-1].replace('-','')
        except:
            new_parcel_id = parcel_id.replace('-','')
        owner_find_url = f'https://www.pcpao.org/general.php?strap={new_parcel_id}'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')
        try:
            try:
                owner_name = str(soup.findAll('td',attrs = {'align':'CENTER'})[2]).split('>')[1].split('<')[0].strip()
            except:
                owner_name = soup.findAll('td',attrs = {'align':'CENTER'})[2].text.strip()
        except:
            owner_name = ''
    elif county == 'lee': 
        owner_find_url = f'https://www.leepa.org/Scripts/PropertyQuery/PropertyQuery.aspx?STRAP={parcel_id}'
        driver.get(owner_find_url)
        try:
            driver.find_element_by_id("btnContinue").click()
            sleep(1)
        except:
            pass
        element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CLASS_NAME, "textPanel"))
                        )
        soup  = bs(driver.page_source,'html.parser')
        try:
            own = soup.findAll('div',attrs  ={'class':'textPanel'})[0].find('div')
            lines = str(own).split('</div>')[0].split('<div>')[1].strip().split('<br/>')
        except:
            pass
        try:
            line_one = lines[0]
        except:
            line_one = ''
        try:
            line_two = lines[1]
        except:
            line_two = ''
        try:
            line_three = lines[2]
        except:
            line_three = ''

        owner_name = ''
        if line_one[0].isnumeric():
            owner_name = owner_name + line_one
            if line_one[-1] == '+':
                owner_name = owner_name +' '+  line_two
            if line_one[-2:] == 'OF':
                owner_name = owner_name +' '+  line_two
            if line_two[-1] == '+':
                owner_name = owner_name + ' ' + line_three
        else:
            owner_name = owner_name + line_one
            if line_one[-1] == '+':
                owner_name = owner_name +' '+  line_two
            if line_one[-2:] == 'OF':
                owner_name = owner_name +' '+  line_two
            if line_two[-1] == '+':
                owner_name = owner_name + ' ' + line_three
        owner_name = owner_name.replace('&amp;','&').strip()

    elif county == 'polk': 
        owner_find_url = f'https://www.polkpa.org/CamaDisplay.aspx?OutputMode=Display&SearchType=RealEstate&Page=FindByID&ParcelID={new_parcel_id}&cookie_test=true'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')
        try:
            try:
                owner_name = soup.find('tr',attrs= {'class':'tr1'}).find('td').text
            except:
                owner_name = ''
            if 'Address 1'==owner_name:
                owner_name = ''
        except:
            owner_name = ''
    elif county == 'manatee': 
        owner_find_url = f'https://www.polkpa.org/CamaDisplay.aspx?OutputMode=Display&SearchType=RealEstate&Page=FindByID&ParcelID={new_parcel_id}&cookie_test=true'
        r = requests.get(owner_find_url,headers = headers)
        soup = bs(r.content,'html.parser')
        try:
            try:
                owner_name = soup.find('tr',attrs= {'class':'tr1'}).find('td').text
            except:
                owner_name = ''
            if 'Address 1'==owner_name:
                owner_name = ''
        except:
            owner_name = ''
    elif county == 'stlucie': 
        url = f'https://paslc.gov/recordcarddata/api/re/PropertyIndex/{parcel_id}'
        try:
            r = requests.get(url,headers =headers)
            soup = bs(r.content,'html.parser')
            acc_num = json.loads(str(soup)).get('AccountNumber')
            url = f'https://paslc.gov/recordcarddata/api/re/baserecordcard/{acc_num}'

            r = requests.get(url,headers =headers)
            soup = bs(r.content,'html.parser')
            owner_name = json.loads(str(soup))[0].get('Owner1')
        except:
            owner_name = ''
    elif county == 'marion': 
        url = f'http://www.pa.marion.fl.us/PRC.aspx?key={parcel_id}&YR=2022&mName=False&mSitus=False'
        try:
            r = requests.get(url,headers =headers)
            soup = bs(r.content,'html.parser')

            owner = soup.findAll('table',attrs = {'style':'width:100%;'})[3].find('tr').text
            owner_name = ''
            for own in owner:
                if own.isnumeric():
                    break
                else:
                    owner_name = owner_name + own
            owner_name = owner_name.replace('More Names','')
        except:
            owner_name = ''

    else:
        owner_name = ''

    datan = {
        'Date': date,
        'Type': types,
        'Auction Status': auction_status,
        'Sold Amount': sold_amount,
        'Sold To': sold_to,
        'Owner Name': owner_name,
        'Auction Type': auction_type,
        'Case #': case_hash,
        'Final Judgement Amount': final_jud_amount,
        'Parcel ID': parcel_id,
        'Property Address': property_address,
        'Assessed Value': assessed_value,
        'Plaintiff Max Bid': plaintiff_max_bid
    }
    listan.append(datan)


def scrape(month,county):
    print(f'Scraping: {month} {county}')
    month_url = f'https://{county}.realforeclose.com/index.cfm?zaction=user&zmethod=calendar&selCalDate=%7Bts%20%27{month}-01%2000%3A00%3A00%27%7D'
    r = requests.get(month_url,headers = headers)
    soup = bs(r.content,'html.parser')

    available_dates = soup.findAll('div',attrs = {'role':'link'})
    s = [str(av['dayid']).split('/')[1] for av in available_dates]
    print(f'Available Dates: {s}\n')

    month_year = str(available_dates[0]['aria-label']).split('-',1)
    month_year = month_year[0] +'-'+ month_year[1].split('-')[-1]
    available_date_urls = [f'https://{county}.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE='+str(av['dayid']) for av in available_dates]
    for url in available_date_urls:
        print(f'Scraping URL: {url}')
        driver.get(url)
        element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "Area_C"))
            )
        soup = bs(driver.page_source,'html.parser')
        try:
            running_aucs = soup.find('div',attrs = {'id':'Area_R'}).findAll('div',recursive=False)
        except:
            running_aucs = []
        try:
            waiting_aucs = soup.find('div',attrs = {'id':'Area_W'}).findAll('div',recursive=False)
        except:
            waiting_aucs = []
        try:
            closed_aucs = soup.find('div',attrs = {'id':'Area_C'}).findAll('div',recursive=False)
        except:
            closed_aucs = []

        if len(closed_aucs)>0:
            for closed in closed_aucs:
                date = url.split('=')[-1]
                types = 'Auctions Closed or Canceled'
                try:
                    left_side = closed.find('div',attrs = {'class':'AUCTION_STATS'})
                    try:
                        right_side = closed.find('div',attrs = {'class':'AUCTION_DETAILS'})
                    except:
                        right_side = closed.find('div',attrs = {'class':'AUCTION_DETAILS adc'})
                    auction_scrape(date,types,left_side,right_side,county)
                except:
                    pass
        if len(running_aucs)>0:
            for running in running_aucs:
                date = url.split('=')[-1]
                types = 'Auctions Running'
                try:
                    left_side = running.find('div',attrs = {'class':'AUCTION_STATS'})
                    try:
                        right_side = running.find('div',attrs = {'class':'AUCTION_DETAILS'})
                    except:
                        right_side = running.find('div',attrs = {'class':'AUCTION_DETAILS adc'})
                    auction_scrape(date,types,left_side,right_side,county)
                except:
                    pass
        if len(waiting_aucs)>0:
            for waiting in waiting_aucs:
                date = url.split('=')[-1]
                types = 'Auctions Waiting'
                try:
                    left_side = waiting.find('div',attrs = {'class':'AUCTION_STATS'})
                    try:
                        right_side = waiting.find('div',attrs = {'class':'AUCTION_DETAILS'})
                    except:
                        right_side = closed.find('div',attrs = {'class':'AUCTION_DETAILS adc'})
                    auction_scrape(date,types,left_side,right_side,county)
                except:
                    pass
    date = url.split('=')[-1].replace('/','-')
    
    df = pd.DataFrame(listan).to_excel(f'{county}_{month_year}.xlsx',encoding = "utf-8-sig",index = False)
    from google.colab import files
    files.download(f'{county}_{month_year}.xlsx')




def main():
    # list_of_county = ['duval','miamidade','broward','hillsborough','orange','pinellas','lee','polk','pasco','manatee','stlucie','marion']
    # count = 1
    # for county in list_of_county:
    #     print(f'{count}. Scrape {county}')
    #     count = count + 1
    # choice = int(input('Enter Choice: ')) - 1
    # print('\n')
    # date_choice = input('Enter Year-Month(Ex. 2022-06): ')
    # scrape(date_choice,list_of_county[choice])
    scrape('2022-07','lee')


main()

Scraping: 2022-07 lee
Available Dates: ['06', '08', '11', '13', '20', '21', '22', '27']

Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/06/2022


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:189: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/08/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/11/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/13/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/20/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/21/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/22/2022
Scraping URL: https://lee.realforeclose.com/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE=07/27/2022


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
owner_find_url = f'https://www.leepa.org/Scripts/PropertyQuery/PropertyQuery.aspx?STRAP=04-48-25-B4-01200.0410'
driver.get(owner_find_url)
try:
    driver.find_element_by_id("btnContinue").click()
    sleep(1)
except:
    pass
element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "textPanel"))
                )
soup  = bs(driver.page_source,'html.parser')
try:
    own = soup.findAll('div',attrs  ={'class':'textPanel'})[0].find('div')
    lines = str(own).split('</div>')[0].split('<div>')[1].strip().split('<br/>')
except:
    pass
try:
    line_one = lines[0]
except:
    line_one = ''
try:
    line_two = lines[1]
except:
    line_two = ''
try:
    line_three = lines[2]
except:
    line_three = ''

owner_name = ''
if line_one[0].isnumeric():
    owner_name = owner_name + line_one
    if line_one[-1] == '+':
        owner_name = owner_name +' '+  line_two
    if line_two[-1] == '+':
        owner_name = owner_name + ' ' + line_three
else:
    owner_name = owner_name + line_one
    if line_one[-1] == '+':
        owner_name = owner_name +' '+  line_two
    if line_two[-1] == '+':
        owner_name = owner_name + ' ' + line_three
owner_name = owner_name.strip()